In [ ]:
import os
R_PATH = "/storage/result/Part2_result"
OUTPUT_PATH = "./output"

NoPressTxt = "NoPress.txt"        
NoResultTxt = "NoResult.txt" 
NoReposTxt = "NoRepo.txt"     
ErrorTxt = "Error.txt" 

RaceCsv = "CVE_analysis_results_RCA.csv"             
ResultCsv = "CVE_analysis_results_RST.csv"        





In [2]:
def get_subdirectories(root_dir):
    return [name for name in os.listdir(root_dir) 
            if os.path.isdir(os.path.join(root_dir, name))]

cve_folder_list = []
parent_folder_list = get_subdirectories(R_PATH)

for parent_folder in parent_folder_list:
    subfolder_path = os.path.join(R_PATH, parent_folder)
    subfolder_list = get_subdirectories(subfolder_path)
    for subfolder in subfolder_list:
        subfolder_full_path = os.path.join(subfolder_path, subfolder)
        cve_folder_list.append(subfolder_full_path)
    print(f"Found {len(subfolder_list)} subfolders in {parent_folder}")

print(len(cve_folder_list))

Found 53 subfolders in savannah
Found 321 subfolders in google
Found 4752 subfolders in github
Found 104 subfolders in openssl
Found 623 subfolders in kernel
Found 70 subfolders in gitlab
Found 154 subfolders in videolan
Found 83 subfolders in other
Found 109 subfolders in freedesktop
6269


In [3]:


with open(os.path.join(OUTPUT_PATH, NoPressTxt), 'w') as f:
    pass
with open(os.path.join(OUTPUT_PATH, NoResultTxt), 'w') as f:
    pass
with open(os.path.join(OUTPUT_PATH, ErrorTxt), 'w') as f:
    pass
with open(os.path.join(OUTPUT_PATH, NoReposTxt), 'w') as f:
    pass
def get_txt_len(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            return len(lines)
    except FileNotFoundError:
        return 0

def check_files(cve_folder):
    files = os.listdir(cve_folder)
    has_error_analysis = "error_analysis.json" in files
    has_root_cause_analysis = "Root_cause_analysis.json" in files
    has_context_preprocess = "context_preprocess.json" in files

    os.makedirs(OUTPUT_PATH, exist_ok=True)
    if not has_context_preprocess:
        repo_file = os.listdir(os.path.join(cve_folder, "old_repos")) if os.path.exists(os.path.join(cve_folder, "old_repos")) else []
        has_repo_info = "info.json" in repo_file
        if not has_repo_info:
            with open(os.path.join(OUTPUT_PATH, NoReposTxt), 'a') as f:
                f.write(f"{cve_folder}\n")
        else:
            with open(os.path.join(OUTPUT_PATH, NoPressTxt), 'a') as f:
                f.write(f"{cve_folder}\n")
    elif not has_error_analysis and not has_root_cause_analysis:
        with open(os.path.join(OUTPUT_PATH, NoResultTxt), 'a') as f:
            f.write(f"{cve_folder}\n")
    elif has_error_analysis and not has_root_cause_analysis:
        with open(os.path.join(OUTPUT_PATH, ErrorTxt), 'a') as f:
            f.write(f"{cve_folder}\n")
    else:
        return True
    return False

valid_cve_folders = []
for cve_folder in cve_folder_list:
    if check_files(cve_folder):
        valid_cve_folders.append(cve_folder)

len_no_repos = get_txt_len(os.path.join(OUTPUT_PATH, NoReposTxt))
len_no_press = get_txt_len(os.path.join(OUTPUT_PATH, NoPressTxt))
len_no_result = get_txt_len(os.path.join(OUTPUT_PATH, NoResultTxt))
len_error = get_txt_len(os.path.join(OUTPUT_PATH, ErrorTxt))
len_valid = len(valid_cve_folders)
print(f"Number of CVE folders with no repo info: {len_no_repos}")
print(f"Number of CVE folders with no press info: {len_no_press}")
print(f"Number of CVE folders with no result info: {len_no_result}")
print(f"Number of CVE folders with error info: {len_error}")
print(f"Number of valid CVE folders: {len_valid}")
assert len_no_repos + len_no_press + len_no_result + len_error + len_valid == len(cve_folder_list), "The total count does not match the number of CVE folders."


Number of CVE folders with no repo info: 147
Number of CVE folders with no press info: 549
Number of CVE folders with no result info: 25
Number of CVE folders with error info: 842
Number of valid CVE folders: 4706


In [4]:

print(f"Norepos / All_CVE: {len_no_repos / len(cve_folder_list):.2%}")
print(f"NoPress / All_Repo_CVE: {len_no_press / (len(cve_folder_list)-len_no_repos):.2%}")
print(f"All CVE Base : {len(cve_folder_list)-len_no_repos-len_no_press}")
ALL_CVE_BASE = len(cve_folder_list) - len_no_repos - len_no_press

Norepos / All_CVE: 2.34%
NoPress / All_Repo_CVE: 8.97%
All CVE Base : 5573


In [5]:
import json
import re

EmptyRootCause = []
ErrorRootCauseTxt = "ErrorRootCause.txt"

def extract_context_preprocess(cve_folder):
    preprocess_file = os.path.join(cve_folder, "context_preprocess.json")
    if not os.path.exists(preprocess_file):
        EmptyRootCause.append(cve_folder)
        return None, []

    try:
        with open(preprocess_file, 'r') as f:
            file_content = f.read().strip()
            if not file_content:  
                EmptyRootCause.append(cve_folder)
                return None, []
                
            content = json.loads(file_content)
            cve_id = str(content[0].get("cve_id", ""))
            cwe_ids = list(content[0].get("cwe_ids", []))
            return cve_id, cwe_ids
    
    except (json.JSONDecodeError, UnicodeDecodeError, KeyError) as e:
        EmptyRootCause.append(cve_folder)
        return None, []


def extract_root_cause_analysis(cve_folder):
    root_cause_file = os.path.join(cve_folder, "Root_cause_analysis.json")
    
    root_cause_file_top10 = os.path.join(cve_folder, "Root_cause_analysis_top10.json")
    root_cause_file_06 = os.path.join(cve_folder, "Root_cause_analysis_06.json")
    if os.path.exists(root_cause_file_06):
        root_cause_file = root_cause_file_06
    if os.path.exists(root_cause_file_top10):
        root_cause_file = root_cause_file_top10

    if not os.path.exists(root_cause_file):
        EmptyRootCause.append(cve_folder)
        return None, {}

    try:
        with open(root_cause_file, 'r') as f:
            file_content = f.read().strip()
            if not file_content: 
                EmptyRootCause.append(cve_folder)
                return None, {}

            content = json.loads(file_content)
            if not isinstance(content, dict) or not content: 
                EmptyRootCause.append(cve_folder)
                return None, {}

            confidence_score = str(content.get("confidence_score", ""))

            tools = {
                "value": 0,
                "func": 0,
                "caller": 0,
                "query": 0,
                "code": 0
            }

            enriched_data = content.get("enriched_data", [])
            for idx, item in enumerate(enriched_data, 1):
                if not isinstance(item, dict):
                    print(f"[DEBUG] Item {idx} in enriched_data is not a dict: {item}")
                    continue

                tool = item.get("tool", "")
                result = item.get("result", "")
              
                if tool.startswith("value_info."):
                    if isinstance(result, str):
                        continue
                    if isinstance(result, list) and result:
                        tools["value"] += 1
                        
                        continue

                    if isinstance(result, dict):
                        struct_def = result.get("struct_definition", "")
                        value_trace = result.get("value_trace", "")
                        if struct_def or value_trace:
                            tools["value"] += 1
                            

                elif tool.startswith("query_info."):
                   
                    if not (isinstance(result, str) and re.search(r"erorr", result, re.IGNORECASE)):
                        tools["query"] += 1
                      

                elif tool.startswith("func_info."):
                    if not (isinstance(result, str) and re.search(r"no valid", result, re.IGNORECASE)):
                        tools["func"] += 1
                       
                elif tool.startswith("caller_info."):
                    if not (isinstance(result, str) and re.search(r"no valid", result, re.IGNORECASE)):
                        tools["caller"] += 1
                        
                elif tool.startswith("code_info."):
                   
                    if not (isinstance(result, str) and re.search(r"error", result, re.IGNORECASE)):
                        tools["code"] += 1
                
            return confidence_score, tools

    except (json.JSONDecodeError, UnicodeDecodeError) as e:
        print(f"[DEBUG] Failed to parse JSON: {e}")
        EmptyRootCause.append(cve_folder)
        return None, {}


In [6]:
import pandas as pd

def generate_csv(valid_cve_folders):
    rac_data = []
    rst_data = []
    for cve_folder in valid_cve_folders:
        cve_id, cwe_ids = extract_context_preprocess(cve_folder)
        confidence_score, tools = extract_root_cause_analysis(cve_folder)
    
        if cve_id is not None and confidence_score is not None and float(confidence_score) >= 0.75:
            if "error_analysis.json" in os.listdir(cve_folder):
                rac_data.append({
                    "cve_id": cve_id,
                    "cwe_ids": ", ".join(cwe_ids),
                    "confidence_score": confidence_score,
                    "value_tool_count": tools["value"],
                    "func_tool_count": tools["func"],
                    "caller_tool_count": tools["caller"],
                    "query_tool_count": tools["query"],
                    "code_tool_count": tools["code"]
                })
            else:
                rst_data.append({
                    "cve_id": cve_id,
                    "cwe_ids": ", ".join(cwe_ids),
                    "confidence_score": confidence_score,
                    "value_tool_count": tools["value"],
                    "func_tool_count": tools["func"],
                    "caller_tool_count": tools["caller"],
                    "query_tool_count": tools["query"],
                    "code_tool_count": tools["code"]
                })
    
    rst_df = pd.DataFrame(rst_data) # only Root_cause_analysis.json
    rac_df = pd.DataFrame(rac_data) # may contain error_analysis.json, but has Root_cause_analysis.json
    rac_df.to_csv(os.path.join(OUTPUT_PATH, RaceCsv), index=False)
    rst_df.to_csv(os.path.join(OUTPUT_PATH, ResultCsv), index=False)
    print(f"Generated {RaceCsv} with {len(rac_data)} entries.")
    print(f"Generated {ResultCsv} with {len(rst_data)} entries.")


generate_csv(valid_cve_folders)

if EmptyRootCause:
    with open(os.path.join(OUTPUT_PATH, ErrorRootCauseTxt), 'w') as f:
        for folder in EmptyRootCause:
            f.write(f"{folder}\n")
    print(f"Empty Root_cause_analysis.json files found in {len(EmptyRootCause)} folders.")


Generated CVE_analysis_results_RCA.csv with 979 entries.
Generated CVE_analysis_results_RST.csv with 3488 entries.
Empty Root_cause_analysis.json files found in 1 folders.


In [7]:
print(f"Total empty root cause folders: {len(EmptyRootCause)}")
print("Total valid CVE folders:", len(valid_cve_folders))


Total empty root cause folders: 1
Total valid CVE folders: 4706


In [8]:
TREE_PATH = "./CWE-TREE"
OUTPUT_PATH = "./output"

import re
import os
import pandas as pd

cwe_tree_path_list = [
    os.path.join(TREE_PATH, filename)
    for filename in os.listdir(TREE_PATH)
    if os.path.isfile(os.path.join(TREE_PATH, filename))
]

def get_cwe_tree_root(cwe_id):
    root = []
    cwe_id = cwe_id.replace("CWE-", "")
    for tree_path in cwe_tree_path_list:
        with open(tree_path, 'r') as f:
            lines = f.readlines()
        
        first_line = lines[0].strip() if lines else ""
        cwe_line = "CWE-" + first_line
        if first_line == cwe_id and cwe_line not in root:
            root.append(cwe_line)
            continue 
        for line in lines[1:]:  
            line = line.strip()
            if re.search(rf"-{re.escape(cwe_id)}\s*$", line) and cwe_line not in root:
                root.append(cwe_line)
                continue  
    
    return root if root else ["Unknown"]


def generate_cwe_roots_csv(CsvFile):
    df = pd.read_csv(os.path.join(OUTPUT_PATH, CsvFile))
    CsvFile = CsvFile.split('.')[0] 
    df['cwe_ids'] = df['cwe_ids'].fillna('')  
    cwe_ids_list = df['cwe_ids'].str.split(',\s*').tolist()

    roots_list = []
    for i, cwe_ids in enumerate(cwe_ids_list):
        roots = []
        for cwe_id in cwe_ids:
            if cwe_id.strip():
        
                root = get_cwe_tree_root(cwe_id.strip())
                roots.append(root)
        roots_list.append(roots)
    df['cwe_roots'] = pd.Series(roots_list)
    df.to_csv(os.path.join(OUTPUT_PATH, CsvFile+"_with_roots.csv"), index=False)
    print("CWE roots added to DataFrame and saved to CSV.")

generate_cwe_roots_csv("CVE_analysis_results_RCA.csv")
generate_cwe_roots_csv("CVE_analysis_results_RST.csv")


CWE roots added to DataFrame and saved to CSV.
CWE roots added to DataFrame and saved to CSV.


In [9]:
import pandas as pd
import os
import ast
from typing import List, Dict, Any

def analyze_cwe_context(file_path_0: str, file_path_1: str) -> None:
    """
    Analyzes CVE data from a CSV file, calculating statistics for CWEs, CWE-roots, and Context.
    
    Args:
        file_path (str): Path to the CSV file containing CVE data
        
    Returns:
        None (prints analysis results to console)
    """
    try:
        df0 = pd.read_csv(file_path_0)
        df1 = pd.read_csv(file_path_1)
        df = pd.concat([df0, df1], ignore_index=True)
    except FileNotFoundError:
        return
    except pd.errors.EmptyDataError:
        return
    except Exception as e:
        print(f"Error reading file: {str(e)}")
        return
    
    # Process CWE IDs and roots
    processed_data = []
    cwe_roots_data = []
    
    for _, row in df.iterrows():
        # Process CWE IDs
        cwe_ids = process_cwe_ids(row['cwe_ids'])
        
        # Process CWE roots
        current_roots = process_cwe_roots(row['cwe_roots'])
        
        # Calculate context score
        context = calculate_context(row)
        
        # Collect data for CWE roots stats
        for root in current_roots:
            cwe_roots_data.append({'cwe_root': root, 'context': context})
        
        # Create entries for each CWE ID
        for cwe_id in cwe_ids:
            entry = {
                'cve_id': row['cve_id'],
                'cwe_id': cwe_id,
                'confidence_score': row['confidence_score'],
                'context': context,
                'cwe_roots': current_roots
            }
            processed_data.append(entry)
    
    # Create DataFrames
    processed_df = pd.DataFrame(processed_data)
    cwe_roots_df = pd.DataFrame(cwe_roots_data)
    
    # Print analysis results
    print("\n=== CVE Context Analysis ===")
    print(f"Total CVE entries suscess processed: {len(df):,}")
    potential_flaws = ALL_CVE_BASE - len(df)
    print(f"Potential flaws in CVE entries: {potential_flaws:,} (if any)\n")
    print(f"All CVE entries may have flaws: {potential_flaws:,} / {ALL_CVE_BASE:,} = {potential_flaws / ALL_CVE_BASE:.2%} ")
    print(f"All CVE entries are valid and processed: {len(df):,} / {ALL_CVE_BASE:,} = {len(df) / ALL_CVE_BASE:.2%} \n")
    
    print_global_stats(processed_df)
    print_cwe_stats(processed_df)
    print_cwe_root_stats(cwe_roots_df)
    
    print("\n=== Analysis Complete ===\n")

def process_cwe_ids(cwe_ids: Any) -> List[str]:
    """Process CWE IDs column into a clean list of CWE IDs"""
    if pd.isna(cwe_ids):
        return ['Unknown_CWE']
    
    try:
        # Handle multiple CWEs separated by commas
        return [cwe.strip() for cwe in str(cwe_ids).split(',')]
    except Exception:
        return ['Unknown_CWE']

def process_cwe_roots(cwe_roots: Any) -> List[str]:
    """Process CWE roots column into a clean list of root CWEs"""
    if pd.isna(cwe_roots):
        return ['Unknown_CWE_Root']
    
    try:
        # Safely evaluate string representation of list
        roots = ast.literal_eval(str(cwe_roots))
        if not isinstance(roots, list):
            return ['Invalid_CWE_Root_Format']
            
        # Flatten nested lists and handle unknown cases
        flat_roots = []
        for item in roots:
            if isinstance(item, list):
                flat_roots.extend(item)
            else:
                flat_roots.append(item)
                
        return [str(root) for root in flat_roots if str(root) != 'nan']
    except (ValueError, SyntaxError):
        return ['Invalid_CWE_Root_Format']

def calculate_context(row: pd.Series) -> int:
    tools = ['value_tool_count', 'func_tool_count', 'caller_tool_count', 
             'query_tool_count', 'code_tool_count']
    return sum(int(row[tool]) if not pd.isna(row[tool]) else 0 for tool in tools)

def print_global_stats(df: pd.DataFrame) -> None:
    print("=== Global Statistics ===")
    print(f"Total CVE -> CWE associations: {len(df):,}")
    print(f"Average confidence score: {df['confidence_score'].mean():.2f}")
    print(f"Average context score: {df['context'].mean():.2f}")
    context_zero_cve_count = df[df['context'] == 0]['cve_id'].nunique()
    print(f"Entries with context = 0: {context_zero_cve_count})")
    print()

def print_cwe_stats(df: pd.DataFrame) -> None:
    print("=== CWE Statistics ===")
    
    cwe_stats = df.groupby('cwe_id').agg(
        count=('cwe_id', 'size'),
        avg_confidence=('confidence_score', 'mean'),
        avg_context=('context', 'mean'),
        max_context=('context', 'max')
    ).reset_index()
    
    cwe_stats['percentage'] = (cwe_stats['count'] / len(df)) * 100
    cwe_stats = cwe_stats.sort_values('count', ascending=False)
    
    # cwe_stats.columns = ['CWE ID', 'Count', 'Avg Confidence', 'Avg Context', 'Max Context', 'Percentage']
    # print(cwe_stats.to_string(float_format="%.2f", index=False))
    # print()

def print_cwe_root_stats(df: pd.DataFrame) -> None:
    """Print statistics about CWE roots including Avg/Max Context"""
    print("=== CWE Root Statistics ===")
    
    root_stats = df.groupby('cwe_root').agg(
        count=('cwe_root', 'size'),
        avg_context=('context', 'mean'),
        max_context=('context', 'max')
    ).reset_index()
    
    root_stats['percentage'] = (root_stats['count'] / len(df)) * 100
    root_stats = root_stats.sort_values('count', ascending=False)
    root_stats['count'] = root_stats['count'].apply(lambda x: f"{x:,}")

    root_stats.columns = ['CWE Root', 'Count', 'Avg Context', 'Max Context', 'Percentage']
    print(root_stats.to_string(float_format="%.2f", index=False))
    total_count = root_stats['Count'].str.replace(',', '').astype(int).sum()
    print(f"\nTotal CWE Roots: {total_count:,}\n")
    print()

if __name__ == "__main__":
    analyze_cwe_context(
        os.path.join(OUTPUT_PATH, "CVE_analysis_results_RCA_with_roots.csv"),
        os.path.join(OUTPUT_PATH, "CVE_analysis_results_RST_with_roots.csv")
    )



=== CVE Context Analysis ===
Total CVE entries suscess processed: 4,467
Potential flaws in CVE entries: 1,106 (if any)

All CVE entries may have flaws: 1,106 / 5,573 = 19.85% 
All CVE entries are valid and processed: 4,467 / 5,573 = 80.15% 

=== Global Statistics ===
Total CVE -> CWE associations: 4,791
Average confidence score: 0.93
Average context score: 3.43
Entries with context = 0: 493)

=== CWE Statistics ===
=== CWE Root Statistics ===
CWE Root Count  Avg Context  Max Context  Percentage
 CWE-664 2,736         3.54           35       52.51
 Unknown   561         3.14           21       10.77
 CWE-707   437         3.19           19        8.39
 CWE-710   336         3.67           24        6.45
 CWE-703   318         3.73           24        6.10
 CWE-682   302         3.70           28        5.80
 CWE-691   248         2.89           20        4.76
 CWE-284   183         3.10           15        3.51
 CWE-693    68         2.75            9        1.31
 CWE-435    16        